In [1]:
from flask import request, jsonify
import base64
import io
from PIL import Image
from ultralytics import YOLO

In [4]:
img = Image.open("zidane.jpg")
# Making it smaller for faster processing
W, H = img.size
max_size = 300
# Calculate the scaling factor
scaling_factor = min(max_size / W, max_size / H)
new_size = (int(W * scaling_factor), int(H * scaling_factor))
img = img.resize(new_size)
W, H = img.size

# Load the model and process the image
model = YOLO("./models/best.onnx", task="segment")
result = model.predict(img)[0]
np_output = result.plot(
    boxes=False,
    color_mode="instance"
)

Loading models/best.onnx for ONNX Runtime inference...

0: 640x640 2 persons, 1554.4ms
Speed: 3.4ms preprocess, 1554.4ms inference, 10.4ms postprocess per image at shape (1, 3, 640, 640)


In [5]:
np_output = np_output.astype("uint8")
np_output

array([[[71, 75, 90],
        [71, 75, 90],
        [74, 78, 92],
        ...,
        [73, 66, 88],
        [66, 62, 84],
        [61, 59, 80]],

       [[41, 48, 74],
        [42, 49, 75],
        [45, 52, 77],
        ...,
        [45, 32, 73],
        [32, 25, 65],
        [23, 19, 59]],

       [[45, 52, 77],
        [46, 53, 79],
        [48, 55, 79],
        ...,
        [48, 33, 80],
        [37, 29, 73],
        [27, 21, 65]],

       ...,

       [[39, 39, 34],
        [37, 36, 33],
        [38, 36, 39],
        ...,
        [54, 54, 41],
        [53, 53, 40],
        [50, 50, 39]],

       [[47, 41, 37],
        [43, 38, 36],
        [44, 36, 40],
        ...,
        [53, 53, 41],
        [53, 53, 39],
        [53, 53, 39]],

       [[49, 40, 37],
        [47, 37, 37],
        [47, 36, 41],
        ...,
        [51, 52, 39],
        [50, 50, 39],
        [49, 49, 38]]], dtype=uint8)

In [7]:
img = Image.fromarray(np_output).convert("RGB")
img.show()